In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sqlalchemy import create_engine
import os


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

%matplotlib inline

dbname = 'Scania - Inbound Logistics'
conn = create_engine('mssql+pyodbc://ADIS2-VM/{}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'.format(dbname),fast_executemany = True) 

Creating categories for matching based on kg and lm uom_type

In [2]:
df = pd.read_sql('select * from dec_transaction',conn)

In [3]:
df.head(2)

,bill_no,bill_date,bill_sender_id,bill_sender_name,load_no,to_no,country_to_consignor,zip_to_consignor,city_to_consignor,name_to_consignor,country_to_consignee,zip_to_consignee,city_to_consignee,name_to_consignee,country_leg_consignor,zip_leg_consignor,city_leg_consignor,name_leg_consignor,country_leg_consignee,zip_leg_consignee,city_leg_consignee,name_leg_consignee,processing_date,pickup_date,weight_to_kg,volume_to_m3,loading_meter_leg,transport_mode_leg,cost_type,cost_desc,cost_rate,amount,uom_type,uom_value,pricing_category
0,27216,10.12.2021,SCAC81,Jan De Rijk,1708024,23020651,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,08.12.2021,08.12.2021,2560.0,78.72,13.6,FTL,SCAPAD,Only Inbound Invoicing Users - Price Adjustment,None,1206.85,not_lm_kg,0.0000,not_lm_kg
1,27034,03.12.2021,SCAC10-13,DSV ES,1695101,30239604,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",29.11.2021,24.11.2021,12845.0,69.75,11.8,LTL,SCAMAT,Maut,23226.8835 kg.,92.90,kg,23226.8835,23000_24499_kg


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   bill_no                341430 non-null  object 
 1   bill_date              341430 non-null  object 
 2   bill_sender_id         341430 non-null  object 
 3   bill_sender_name       341430 non-null  object 
 4   load_no                341430 non-null  int64  
 5   to_no                  341430 non-null  object 
 6   country_to_consignor   341430 non-null  object 
 7   zip_to_consignor       341430 non-null  object 
 8   city_to_consignor      341430 non-null  object 
 9   name_to_consignor      341430 non-null  object 
 10  country_to_consignee   341430 non-null  object 
 11  zip_to_consignee       341430 non-null  object 
 12  city_to_consignee      341430 non-null  object 
 13  name_to_consignee      341430 non-null  object 
 14  country_leg_consignor  341430 non-nu

In [5]:
#df['uom_value'] = pd.to_numeric(df['uom_value'],errors='coerce')

In [5]:
df.groupby("uom_type", as_index=False).agg(LoadCount=('load_no','nunique'),
                                     ToCount=('to_no','nunique'),
                                     TotalPaid=('amount','sum'),
                                     v=('uom_value','sum')).sort_values(by='TotalPaid',ascending=False)

,uom_type,LoadCount,ToCount,TotalPaid,v
1,lm,8570,49728,9848744.59,1.028397e+06
2,not_lm_kg,26671,73537,3312973.03,0.000000e+00
0,kg,13917,40749,2771097.36,3.672159e+08


In [6]:
df['uom_value'] = df['uom_value'].fillna(0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   bill_no                341430 non-null  object 
 1   bill_date              341430 non-null  object 
 2   bill_sender_id         341430 non-null  object 
 3   bill_sender_name       341430 non-null  object 
 4   load_no                341430 non-null  int64  
 5   to_no                  341430 non-null  object 
 6   country_to_consignor   341430 non-null  object 
 7   zip_to_consignor       341430 non-null  object 
 8   city_to_consignor      341430 non-null  object 
 9   name_to_consignor      341430 non-null  object 
 10  country_to_consignee   341430 non-null  object 
 11  zip_to_consignee       341430 non-null  object 
 12  city_to_consignee      341430 non-null  object 
 13  name_to_consignee      341430 non-null  object 
 14  country_leg_consignor  341430 non-nu

In [8]:
def categorize_uom(row):
    if row['uom_type'] == 'kg':
        if row['uom_value'] <= 49:
            return '0_49_kg'
        elif row['uom_value'] <= 99:
            return '50_99_kg'
        elif row['uom_value'] <= 149:
            return '100_149_kg'
        elif row['uom_value'] <= 199:
            return '150_199_kg'
        elif row['uom_value'] <= 299:
            return '200_299_kg'
        elif row['uom_value'] <= 399:
            return '300_399_kg'
        elif row['uom_value'] <= 499:
            return '400_499_kg'
        elif row['uom_value'] <= 599:
            return '500_599_kg'
        elif row['uom_value'] <= 699:
            return '600_699_kg'
        elif row['uom_value'] <= 799:
            return '700_799_kg'
        elif row['uom_value'] <= 899:
            return '800_899_kg'
        elif row['uom_value'] <= 999:
            return '900_999_kg'
        elif row['uom_value'] <= 1099:
            return '1000_1099_kg'
        elif row['uom_value'] <= 1199:
            return '1100_1199_kg'
        elif row['uom_value'] <= 1299:
            return '1200_1299_kg'
        elif row['uom_value'] <= 1399:
            return '1300_1399_kg'
        elif row['uom_value'] <= 1499:
            return '1400_1499_kg'
        elif row['uom_value'] <= 1749:
            return '1500_1749_kg'
        elif row['uom_value'] <= 1999:
            return '1750_1999_kg'
        elif row['uom_value'] <= 2249:
            return '2000_2249_kg'
        elif row['uom_value'] <= 2499:
            return '2250_2499_kg'
        elif row['uom_value'] <= 2999:
            return '2500_2999_kg'
        elif row['uom_value'] <= 3499:
            return '3000_3499_kg'
        elif row['uom_value'] <= 3999:
            return '3500_3999_kg'
        elif row['uom_value'] <= 4499:
            return '4000_4499_kg'
        elif row['uom_value'] <= 4999:
            return '4500_4999_kg'
        elif row['uom_value'] <= 5999:
            return '5000_5999_kg'
        elif row['uom_value'] <= 6999:
            return '6000_6999_kg'
        elif row['uom_value'] <= 7999:
            return '7000_7999_kg'
        elif row['uom_value'] <= 8999:
            return '8000_8999_kg'
        elif row['uom_value'] <= 9999:
            return '9000_9999_kg'
        elif row['uom_value'] <= 10999:
            return '10000_10999_kg'
        elif row['uom_value'] <= 11999:
            return '11000_11999_kg'
        elif row['uom_value'] <= 12999:
            return '12000_12999_kg'
        elif row['uom_value'] <= 13999:
            return '13000_13999_kg'
        elif row['uom_value'] <= 14999:
            return '14000_14999_kg'
        elif row['uom_value'] <= 15999:
            return '15000_15999_kg'
        elif row['uom_value'] <= 16999:
            return '16000_16999_kg'
        elif row['uom_value'] <= 17999:
            return '17000_17999_kg'
        elif row['uom_value'] <= 18999:
            return '18000_18999_kg'
        elif row['uom_value'] <= 19999:
            return '19000_19999_kg'
        elif row['uom_value'] <= 21499:
            return '20000_21499_kg'
        elif row['uom_value'] <= 22999:
            return '21500_22999_kg'
        elif row['uom_value'] <= 24499:
            return '23000_24499_kg'
        elif row['uom_value'] > 24499:
            return 'FTL_kg'

# type of cost calculation
df["pricing_category"] = df.apply(categorize_uom, axis=1)

In [10]:
df[df['uom_type']=='kg'].head()

,bill_no,bill_date,bill_sender_id,bill_sender_name,load_no,to_no,country_to_consignor,zip_to_consignor,city_to_consignor,name_to_consignor,country_to_consignee,zip_to_consignee,city_to_consignee,name_to_consignee,country_leg_consignor,zip_leg_consignor,city_leg_consignor,name_leg_consignor,country_leg_consignee,zip_leg_consignee,city_leg_consignee,name_leg_consignee,processing_date,pickup_date,weight_to_kg,volume_to_m3,loading_meter_leg,transport_mode_leg,cost_type,cost_desc,cost_rate,amount,uom_type,uom_value,pricing_category
1,27034,03.12.2021,SCAC10-13,DSV ES,1695101,30239604,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",29.11.2021,24.11.2021,12845.0,69.750,11.800,LTL,SCAMAT,Maut,23226.8835 kg.,92.9,kg,23226.8835,23000_24499_kg
2,27034,03.12.2021,SCAC10-13,DSV ES,1695100,30239615,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,29.11.2021,24.11.2021,13812.0,72.853,13.262,LTL,SCAMAT,Maut,24499.0 kg.,92.9,kg,24499.0000,23000_24499_kg
3,27034,03.12.2021,SCAC10-13,DSV ES,1700008,30240209,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,03.12.2021,30.11.2021,14772.0,71.332,13.600,LTL,SCAMAT,Maut,24499.0 kg.,92.9,kg,24499.0000,23000_24499_kg
4,27034,03.12.2021,SCAC10-13,DSV ES,1699921,30240157,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,03.12.2021,30.11.2021,8654.0,75.361,13.304,LTL,SCAMAT,Maut,24499.0 kg.,92.9,kg,24499.0000,23000_24499_kg
5,27034,03.12.2021,SCAC10-13,DSV ES,1699922,30240166,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,50290,EPILA ZARAGOZA,KONGSBERG ACTUATION SYSTEM S.L.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,50290,EPILA ZARAGOZA,KONGSBERG ACTUATION SYSTEM S.L.,03.12.2021,30.11.2021,18809.0,76.086,13.600,LTL,SCAMAT,Maut,24499.0 kg.,92.9,kg,24499.0000,23000_24499_kg


In [11]:
def categorize_uom_lm(row):
    if row['uom_type'] == 'lm':
        if row['uom_value'] <= 0.5:
            return '0.5_lm'
        elif row['uom_value'] <= 1:
            return '1_lm'
        elif row['uom_value'] <= 2:
            return '2_lm'
        elif row['uom_value'] <= 3:
            return '3_lm'
        elif row['uom_value'] <= 4:
            return '4_lm'
        elif row['uom_value'] <= 5:
            return '5_lm'
        elif row['uom_value'] <= 6:
            return '6_lm'
        elif row['uom_value'] <= 7:
            return '7_lm'
        elif row['uom_value'] <= 8:
            return '8_lm'
        elif row['uom_value'] <= 9:
            return '9_lm'
        elif row['uom_value'] <= 10:
            return '10_lm'
        elif row['uom_value'] <= 11:
            return '11_lm'
        elif row['uom_value'] <= 12:
            return '12_lm'
        elif row['uom_value'] <= 13.6:
            return '13.6_lm'
        elif row['uom_value'] > 13.6:
            return 'FTL_lm'
    elif row['uom_type'] == 'kg':
        return row["pricing_category"]
    else:
        return 'not_lm_kg'
        
        
        
# type of cost calculation
df["pricing_category"] = df.apply(categorize_uom_lm, axis=1)

In [12]:
#df["pricing_category"].value_counts()
#[df['uom_type']=='lm']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   bill_no                341430 non-null  object 
 1   bill_date              341430 non-null  object 
 2   bill_sender_id         341430 non-null  object 
 3   bill_sender_name       341430 non-null  object 
 4   load_no                341430 non-null  int64  
 5   to_no                  341430 non-null  object 
 6   country_to_consignor   341430 non-null  object 
 7   zip_to_consignor       341430 non-null  object 
 8   city_to_consignor      341430 non-null  object 
 9   name_to_consignor      341430 non-null  object 
 10  country_to_consignee   341430 non-null  object 
 11  zip_to_consignee       341430 non-null  object 
 12  city_to_consignee      341430 non-null  object 
 13  name_to_consignee      341430 non-null  object 
 14  country_leg_consignor  341430 non-nu

In [13]:
df[df['uom_type']=='not_lm_kg'][['uom_type','pricing_category']].value_counts()


uom_type   pricing_category
not_lm_kg  not_lm_kg           200568
dtype: int64

In [15]:
df2 = df.groupby(['load_no','cost_type'],as_index=False).agg(tr_mode_Type = ('transport_mode_leg','nunique'),
                         uomType_co = ('uom_type','nunique'),
                         uomvalue_co = ('uom_value','nunique'))




df2[df2['tr_mode_Type']>1]

,load_no,cost_type,tr_mode_Type,uomType_co,uomvalue_co


---
Creating the groups as per grouping structure layout, preparing alternative grouping methods 
1. grouping based on loadNo, country from, city from, country to, city to, transporter and then break it down by cost
2. Unpivoting the countries and cities

In [16]:
# df.groupby(['Load no','Country TO consignor','City TO consignor','Country TO consignee','City TO consignee','Cost desc'], 
#            as_index=False).agg(amount=('Amount','sum'))

In [17]:
count_from = df.groupby(['load_no'],as_index=False).agg(
                    From_country=('country_to_consignor','nunique')
                    ,From_city=('city_to_consignor','nunique'))


count_to = df.groupby(['load_no'],as_index=False).agg(
                    To_country=('country_to_consignee','nunique')
                    ,To_city=('city_to_consignee','nunique'))

In [27]:
df_g = df.groupby(['load_no','country_to_consignor','city_to_consignor','cost_desc'], 
           as_index=False).agg(
           To_country=('country_to_consignee','unique'),
           To_city=('city_to_consignee','unique'),
           
           leg_consignor_city=('city_leg_consignor','unique'),
           leg_consignor_name=('name_leg_consignor','unique'),    
           leg_consignee_city=('city_leg_consignee','unique'),
           leg_consignee_name=('name_leg_consignee','unique'),    
    
    
    
           Transpot_count=('to_no','nunique'),
           Bill_count=('bill_no','nunique'),
           transporter=('bill_sender_id','max'),
           transporter_name=('bill_sender_name','max'),
           pickup_date=('pickup_date','max'),
          weight_kg=('weight_to_kg','unique'),
          volumn_m3=('volume_to_m3','unique'),
          loadind_meter=('loading_meter_leg','unique'),
           transport_mode=('transport_mode_leg','max'),
           pricing_category=('pricing_category','max'),
           uom_type=('uom_type','max'),
           uom_value=('uom_value','unique'),
           amount=('amount','sum'))

---
2. Unpivoting the countries and cities, highlighting loads numbers delivered to multi destinations

---

In [34]:
# to country column
df_split = pd.concat([df_g, pd.DataFrame(df_g['To_country'].to_list())], axis=1)

df_split.columns= ['load_no', 'country_to_consignor', 'city_to_consignor',
       'cost_desc', 'To_country', 'To_city', 'leg_consignor_city',
       'leg_consignor_name', 'leg_consignee_city', 'leg_consignee_name',
       'Transpot_count', 'Bill_count', 'transporter', 'transporter_name',
       'pickup_date', 'weight_kg', 'volumn_m3', 'loadind_meter',
       'transport_mode', 'pricing_category', 'uom_type', 'uom_value',
       'amount','dist_country_1', 'dist_country_2','dist_country_3', 'dist_country_4', 'dist_country_5']

# to city column
df_final = pd.concat([df_split,pd.DataFrame(df_split['To_city'].to_list())], axis=1)

df_final.columns= ['load_no', 'country_to_consignor', 'city_to_consignor',
       'cost_desc', 'To_country', 'To_city', 'leg_consignor_city',
       'leg_consignor_name', 'leg_consignee_city', 'leg_consignee_name',
       'Transpot_count', 'Bill_count', 'transporter', 'transporter_name',
       'pickup_date', 'weight_kg', 'volumn_m3', 'loadind_meter',
       'transport_mode', 'pricing_category', 'uom_type', 'uom_value',
       'amount', 'dist_country_1', 'dist_country_2', 'dist_country_3',
       'dist_country_4', 'dist_country_5','dist_city_1', 'dist_city_2', 'dist_city_3', 'dist_city_4', 'dist_city_5',
         'dist_city_6', 'dist_city_7', 'dist_city_8', 'dist_city_9', 'dist_city_10', 'dist_city_11']

In [35]:
df_final = df_final.drop(columns=['To_country','To_city'])

In [36]:
df_final = df_final.fillna(value=0)

In [37]:
df_final['dist_country_count'] = df_final[['dist_country_1', 'dist_country_2', 
                                     'dist_country_3','dist_country_4','dist_country_5']].astype(bool).sum(axis=1)



df_final['dist_city_count'] = df_final[['dist_city_1', 'dist_city_2', 'dist_city_3','dist_city_4',
                                 'dist_city_5','dist_city_6','dist_city_7','dist_city_8','dist_city_9',
                                 'dist_city_10','dist_city_11']].astype(bool).sum(axis=1)

**Thoughts and issues**

for example: 1706034, 1706033, 1693936

- 1706034 costs doesn't match
- Some loads cost seems very unrealistic such as 1564732 (there are many like that)
- it seems that we need to several way of grouping to obtain the loads costs. Considering that different type of transfer defines which pricing file to use (pre-load, FTL/LTL...etc)
    - Type of transfer could be defined by name_leg_consignor and consignee as they are the one showing x-docks or not
- Will add column in SQL defining: 
    - if it comes from x-dock then pre-load
    - if it comes from production unit then component flow
    - ...etc

In [44]:
df_g[df_g['load_no']==1564732]['amount'].sum()

-50.70999999999996

In [23]:
df_g[(df_g['load_no']==1706034) & (df_g['cost_desc']=='Freight Charges') ]

,load_no,country_to_consignor,city_to_consignor,cost_desc,To_country,To_city,leg_consignor_city,leg_consignor_name,leg_consignee_city,leg_consignee_name,Transpot_count,Bill_count,transporter,transporter_name,pickup_date,weight_kg,volumn_m3,loadind_meter,transport_mode,pricing_category,uom_type,uom_value,amount
79986,1706034,TR,BURSA,Freight Charges,PL,[SLUPSK],ISTANBUL,EKOL (TR) X-DOCK GEBZE,HERNE,DUVENBECK (DE) X-DOCK HERNE,1,2,SCAC31-1,EKOL,08.12.2021,[175.0],[0.576],[0.1],MR,150_199_kg,lm,"[191.808, 10.266]",24.07
79990,1706034,TR,BZE/KOCAELI,Freight Charges,PL,[SLUPSK],ISTANBUL,EKOL X-DOCK GEBZE,HERNE,DUVENBECK (DE) X-DOCK HERNE,2,2,SCAC31-1,EKOL,08.12.2021,"[2385.0, 1418.0]","[18.06, 12.03]","[5.219, 3.513]",MR,9000_9999_kg,lm,"[9655.15, 6499.05, 10.266]",1976.34
79994,1706034,TR,CAYIROVA-GEBZE,Freight Charges,NL,[Hasselt],ISTANBUL,EKOL X-DOCK GEBZE,Hasselt,Scania CV AB,1,1,SCAC31-1,EKOL,08.12.2021,[142.0],[1.921],[0.272],MR,11_lm,lm,[10.266],74.85
79998,1706034,TR,GEBZE - KOCAELI,Freight Charges,NL,[Hasselt],ISTANBUL,EKOL X-DOCK GEBZE,Hasselt,Scania CV AB,1,1,SCAC31-1,EKOL,08.12.2021,[1356.0],[3.236],[0.487],MR,11_lm,lm,[10.266],158.66
80002,1706034,TR,KOCAELI,Freight Charges,NL,[Hasselt],ISTANBUL,EKOL (TR) X-DOCK GEBZE,Hasselt,Scania CV AB,1,1,SCAC31-1,EKOL,08.12.2021,[1782.0],[1.848],[0.275],MR,11_lm,lm,[10.266],208.50
80006,1706034,TR,RSA,Freight Charges,PL,[SLUPSK],ISTANBUL,EKOL X-DOCK GEBZE,HERNE,DUVENBECK (DE) X-DOCK HERNE,1,2,SCAC31-1,EKOL,08.12.2021,[150.0],[1.44],[0.4],MR,700_799_kg,lm,"[740.0, 10.266]",90.53
